# HOME ASSIGNMENT #3: SLACK API - TO GSHEET
**Mục đích của bài Assignment**
- Lấy thông tin các Users từ Slack của DataCracy (BTC, Mentors và Learners)
- `**[Optional 1]**` Đưa danh sách Users lên Google Spreadsheet, để theo dõi 
- `**[Optional 2]**` Lấy thông tin Assignment Submission và số Reviews trên `#atom-assignmentnt2` và cập nhật lên Spreadsheet, để theo dõi các học viên đã nộp bài và được review

**Các kiến thức sẽ áp dụng**
- Ôn lại và luyện tập thêm về concept API (cụ thể sử dụng API Slack)
- Trích xuất thông tin từ JSON
- Dùng module gspread để đưa thông tin lên Google Spreadsheet

## 0. Load Modules

In [2]:
pip install gspread

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install gspread_dataframe

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install oauth2client

Note: you may need to restart the kernel to use updated packages.


In [22]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

## 1. Slack API: User List
* Bạn có thể đọc lại về concept API [HERE](https://anhdang.gitbook.io/datacracy/atom/3-data-tools-2/3.2-spotify-api-and-postman)
* Assignment này sẽ dùng Slack API để lấy thông tin về Learners và theo dõi các bài tập đã nộp và được review (sau đó cập nhật lên Google Spreadsheet)
* ===> **NOTICE**: Slack API authorize bằng Bearer Token `xoxb-...-...-...` (Sẽ được cung cấp riêng)
* Update file `env_variable.json` như trong [Assignment#2](../assignment_2/home_assignment_2.ipynb)
* ==> Nếu bạn dùng Google Colab, upload file vào Colab ([Hướng dẫn](https://colab.research.google.com/notebooks/io.ipynb))

In [2]:
!dir

 Volume in drive F is Data
 Volume Serial Number is 58AE-208F

 Directory of f:\Google Drive\Colab Notebooks\Assignment2\atom-assignments\assignment_3

21-May-21  02:32 PM    <DIR>          .
21-May-21  02:32 PM    <DIR>          ..
21-May-21  02:32 PM                89 env_variable.json
21-May-21  01:08 PM            29,535 home_assignment_3.ipynb
21-May-21  01:08 PM               238 README.md
               3 File(s)         29,862 bytes
               2 Dir(s)  179,005,333,504 bytes free


In [3]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

In [4]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN'] 

In [10]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

### TODO #1
Hoàn tất đoạn code sau

In [15]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [16]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [17]:
user_df[user_df.display_name == 'MAD'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

,user_id,name,display_name,real_name,title,phone,is_bot
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False


-------------- HẾT PHẦN BẮT BUỘC ---------------------

## Option 1: Update data => Google SpreadSheet

### TODO#2
Tạo service account (output là file json), file này để cho phép ta access vào Google Spreadsheet:

1. Làm theo hướng dẫn: [Google Create a Service Account](https://support.google.com/a/answer/7378726?hl=en)
![google_service_account](../img/google_service_account.png)
2. Lưu file JSON (chứa credential về máy)
![gservice_acc_json](../img/gservice_acc_json.png)
3. Nhớ Enable [Google Drive API](https://console.cloud.google.com/marketplace/product/google/drive.googleapis.com?q=search&referrer=search&project=quickstart-313303) (Nếu bạn chạy code báo lỗi chưa enable API thì vào link trong phần lỗi để Enable, sau khi kích hoạt có thể cần vài phút để chạy được)
![enable_api](../img/enable_api.png)
* ==> Upload file Gsheet Credential JSON nếu bạn dùng Colab 
* ==> Nếu bạn để key trong repo git, **NHỚ** để file json vào `.gitignore` để không bị leaked key)


In [18]:
!dir

 Volume in drive F is Data
 Volume Serial Number is 58AE-208F

 Directory of f:\Google Drive\Colab Notebooks\Assignment2\atom-assignments\assignment_3

21-May-21  03:02 PM    <DIR>          .
21-May-21  03:02 PM    <DIR>          ..
21-May-21  03:02 PM             2,326 clear-apogee-265809-1309d5ecf61d.json
21-May-21  02:32 PM                89 env_variable.json
21-May-21  01:08 PM            29,535 home_assignment_3.ipynb
21-May-21  01:08 PM               238 README.md
               4 File(s)         32,188 bytes
               2 Dir(s)  179,005,325,312 bytes free


In [24]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'clear-apogee-265809-1309d5ecf61d.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


**Tạo Spreadsheet**

1. Tạo Spreadsheet trên google
2. Invite account trong `client_email` (file JSON Gsheet Credential bên trên) vào Spreadsheet (quyền Editor)
![enable_api](../img/enable_api.png)
3. Lấy `SPREADSHEET_KEY` (nằm trong chính URL của Spreadhstee): `https://docs.google.com/spreadsheets/d/<SPREADSHEET_KEY>/edit#gid=0`

![add_gsheet](../img/add_gsheet.png)

In [27]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '19ctZvMgvT2ydbnP3QXmPrSm2p70B-R0_SWVgqTouhXw' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

![slack_user_gsheet](../img/slack_user_gsheet.png)

-------------- HẾT PHẦN OPTION 1 ---------------------

## Option 2: Ai đã nộp bài?


### Slack API: Channel List

In [29]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']

In [30]:
channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'id': 'C01B4PVGLVB',
 'name': 'general',
 'is_channel': True,
 'is_group': False,
 'is_im': False,
 'created': 1600856703,
 'is_archived': False,
 'is_general': True,
 'unlinked': 0,
 'name_normalized': 'general',
 'is_shared': False,
 'parent_conversation': None,
 'creator': 'U01BE2PR6LU',
 'is_ext_shared': False,
 'is_org_shared': False,
 'shared_team_ids': ['T01B7SGGMLJ'],
 'pending_shared': [],
 'pending_connected_team_ids': [],
 'is_pending_ext_shared': False,
 'is_member': False,
 'is_private': False,
 'is_mpim': False,
 'topic': {'value': '', 'creator': '', 'last_set': 0},
 'purpose': {'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.',
  'creator': 'U01BE2PR6LU',
  'last_set': 1600856703},
 'previous_names': [],
 'num_members': 63}

### TODO#3 
* Tìm id của channel #atom-assignment2

In [50]:
for i in range(len(channel_ls)): 
    if(channel_ls[i]['name']=='atom-assignment2'): print(channel_ls[i]['id'])

C021FSDN7LJ


### Slack API: List messages trong 1 channel

In [51]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C01U6P7LZ8F"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [52]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [199]:
msg_ls[21]

{'attachments': [{'fallback': 'Google Colaboratory',
   'from_url': 'https://colab.research.google.com/drive/1VnMQdhpjKldwvJnjqpYO0sQkeaVbvNyB?usp=sharing',
   'id': 1,
   'image_bytes': 4534,
   'image_height': 250,
   'image_url': 'https://colab.research.google.com/img/colab_favicon_256px.png',
   'image_width': 250,
   'original_url': 'https://colab.research.google.com/drive/1VnMQdhpjKldwvJnjqpYO0sQkeaVbvNyB?usp=sharing',
   'service_icon': 'https://colab.research.google.com/img/favicon.ico?vrz=colab-20210422-060049-RC00_369841174',
   'service_name': 'colab.research.google.com',
   'title': 'Google Colaboratory',
   'title_link': 'https://colab.research.google.com/drive/1VnMQdhpjKldwvJnjqpYO0sQkeaVbvNyB?usp=sharing'}],
 'blocks': [{'block_id': '5uv',
   'elements': [{'elements': [{'text': 'Mình gửi bài assignment1:\nTask 1: ',
       'type': 'text'},
      {'type': 'link',
       'url': 'https://docs.google.com/spreadsheets/d/1XDHiv72ITLoY1aGYuc1LbqpVq20k72th4XRYPmqSAhI/edit?usp=sh

In [53]:
not_learners_id = ['U01BE2PR6LU']

In [67]:
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(20):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: github = github_link[0]
      if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
      if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
      if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
      print(ts, user, reply_users_count, reply_users, latest_reply, github)

2021-05-10 11:51:46.004700 U01UJ9LG5U5 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 16:14:31.003600 https://github.com/danhpcv/
2021-05-10 11:35:39.003800 U01UMC08KL2 2 ['U01UTGS5ZNY', 'U01UMC08KL2'] 2021-05-18 15:09:59.000800 https://github.com/auslynnguyen/
2021-05-08 13:31:59.005200 U01UMJ36QD9 1 ['U01UMBX2X18'] 2021-05-08 13:36:43.006000 https://github.com/auslynnguyen/
2021-05-04 20:55:46.001800 U01UTGVPE7N 1 ['U01UMBX2X18'] 2021-05-08 13:36:43.006000 https://github.com/hoaintp/


### TODO#4
* Tạo thành 1 bảng chứa các thông tin trên và update lên Spreadsheet (Sheet: Assignment#2 Submission)

In [68]:
git_dict = {'time_stampt':[], 'user':[], 'reply_users_count':[],'reply_users':[],'latest_reply':[],'github':[]}
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(20):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: github = github_link[0]
      if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
      if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
      if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
      git_dict['time_stampt'].append(ts)
      git_dict['user'].append(user)
      git_dict['reply_users_count'].append(reply_users_count)
      git_dict['reply_users'].append(reply_users)
      git_dict['latest_reply'].append(latest_reply)
      git_dict['github'].append(github)


In [70]:
git_df = pd.DataFrame(git_dict)
sheet_index_no = 1
spreadsheet_key = '19ctZvMgvT2ydbnP3QXmPrSm2p70B-R0_SWVgqTouhXw' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, git_df)

-------------- HẾT PHẦN OPTION 2 ---------------------